In [190]:
import pandas as pd
import numpy as np

In [191]:
# Importing Data
from data import Dream_report, df , stock_info
# Import data logic model revenue prediction
#from main import overall_predicted_labels # 只有方向，要把ddate, stockid補上
from DreamReport import logit_direction

In [192]:
#type(Dream_report['stockid'][0])
logit_direction['stockid'] = logit_direction['stockid'].astype(str)
logit_direction['predict_labels'] = logit_direction['predict_labels'].astype(float)
#print(logit_direction)

In [193]:
merged = pd.merge(logit_direction, Dream_report, on=['ddate', 'stockid'])
# Filter rows where 'predcited_labels' are different #兩模型預測相反的結果
result = merged[merged['predict_labels'] != merged['direction']]
#print(result)

In [194]:
print(result[result['ddate'] == 202108])

        ddate stockid  predict_labels  direction
45886  202108    1102             0.0        1.0
45887  202108    1103             0.0        1.0
45894  202108    1210             0.0        1.0
45896  202108    1215             0.0        1.0
45897  202108    1216             0.0        1.0
...       ...     ...             ...        ...
47615  202108    9945             0.0        1.0
47617  202108    9949             1.0        0.0
47621  202108    9958             0.0        1.0
47622  202108    9960             0.0        1.0
47623  202108    9962             1.0        0.0

[813 rows x 4 columns]


In [195]:
result['year'] = result['ddate'].astype(str).str[:4]

# Count the number of occurences for each year
year_counts = result['year'].value_counts()

print(year_counts)

year
2021    7832
2020    7410
2022    6859
2017    6714
2018    6709
2016    6706
2023    6429
2019    6410
Name: count, dtype: int64


C:\Users\user1\AppData\Local\Temp\ipykernel_9852\891335564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['ddate'].astype(str).str[:4]


In [196]:
result['month'] = result['ddate'].astype(str).str[4:6]

# Count the number of occurences for each month
month_counts = result['month'].value_counts()

print(month_counts)

month
08    5511
04    5152
10    5130
09    5130
06    4996
07    4938
05    4923
11    4381
02    4156
12    4138
01    3788
03    2826
Name: count, dtype: int64


C:\Users\user1\AppData\Local\Temp\ipykernel_9852\3621829736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['month'] = result['ddate'].astype(str).str[4:6]


In [197]:
march_rows = result[result['month'] == '03']

march_counts = march_rows['ddate'].value_counts()

print(march_counts)

ddate
202303    460
202003    426
202203    401
202103    396
201703    370
201603    287
201803    260
201903    226
Name: count, dtype: int64


In [198]:
aug_rows = result[result['month'] == '08']

aug_counts = aug_rows['ddate'].value_counts()

print(aug_counts)

ddate
202108    813
202308    748
201808    721
201908    675
202008    666
201608    639
202208    631
201708    618
Name: count, dtype: int64


In [199]:
date_counts = result['ddate'].value_counts()

print(date_counts)

ddate
202108    813
202110    763
202304    753
202011    751
202308    748
         ... 
202103    396
201703    370
201603    287
201803    260
201903    226
Name: count, Length: 94, dtype: int64


In [200]:
origin_counts = merged['ddate'].value_counts()
#print(origin_counts)

merged['Y'] = merged['ddate'].astype(str).str[:4]
origin_Ycounts = merged['Y'].value_counts()
print(origin_Ycounts)

merged['M'] = merged['ddate'].astype(str).str[4:6]
origin_Mcounts = merged['M'].value_counts()
print(origin_Mcounts) 

Y
2022    21105
2021    20816
2020    20446
2019    19990
2018    19566
2017    19080
2016    18364
2023    17799
Name: count, dtype: int64
M
10    13492
09    13466
08    13440
07    13406
06    13363
05    13336
04    13313
03    13305
02    13283
01    13276
12    11766
11    11720
Name: count, dtype: int64


In [201]:
# "每個月"兩模型預測相反的個股數 / 兩模型預測的總個股數
month_counts/origin_Mcounts*100

01    28.532691
02    31.288113
03    21.240135
04    38.699016
05    36.915117
06    37.386814
07    36.834253
08    41.004464
09    38.095945
10    38.022532
11    37.380546
12    35.169131
Name: count, dtype: float64

In [202]:
# "每年"兩模型預測相反的個股數 / 兩模型預測的總個股數
print(year_counts/origin_Ycounts*100)
max_date = (date_counts/origin_counts*100).idxmax()
min_date = (date_counts/origin_counts*100).idxmin()

# 預測相反比例最大、最小的日期
print(max_date)
print(min_date)

2016    36.517099
2017    35.188679
2018    34.289073
2019    32.066033
2020    36.241808
2021    37.624904
2022    32.499408
2023    36.120007
Name: count, dtype: float64
202108
201903


In [203]:
stock_info.rename(columns={'股票代號':'stockid', '指數彙編分類代號':'classi_code'}, inplace=True)
# 刪除重複個股資訊
stock_info_dedup = stock_info.drop_duplicates(subset='stockid')
# 將個股資訊加至result df
pred_merge_code = result.merge(stock_info_dedup, on='stockid', how='left')

In [204]:
print(pred_merge_code[pred_merge_code['ddate']==202108])

        ddate stockid  predict_labels  direction  year month 中文簡稱 上市上櫃  \
16202  202108    1102             0.0        1.0  2021    08   亞泥    1   
16203  202108    1103             0.0        1.0  2021    08   嘉泥    1   
16204  202108    1210             0.0        1.0  2021    08   大成    1   
16205  202108    1215             0.0        1.0  2021    08   卜蜂    1   
16206  202108    1216             0.0        1.0  2021    08   統一    1   
...       ...     ...             ...        ...   ...   ...  ...  ...   
17010  202108    9945             0.0        1.0  2021    08  潤泰新    1   
17011  202108    9949             1.0        0.0  2021    08   琉園    2   
17012  202108    9958             0.0        1.0  2021    08  世紀鋼    1   
17013  202108    9960             0.0        1.0  2021    08  邁達康    2   
17014  202108    9962             1.0        0.0  2021    08   有益    2   

      classi_code   指數彙編分類  
16202      C11010    傳產-水泥  
16203      C11010    傳產-水泥  
16204      C12010    傳產-

In [205]:
# 計算各種產業兩個模型判斷相反的次數，並降序排列
count_classi = pred_merge_code.groupby(['classi_code', '指數彙編分類']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
print(count_classi)

   classi_code          指數彙編分類  counts
7       C17020           傳產-生技    4347
67      C29020           傳產-其他    3932
14      C23010      電子上游-IC-設計    3101
59      C25010           傳產-營建    2719
4       C15010           傳產-電機    2039
..         ...             ...     ...
68      C29030          傳產-自行車     152
53      C23410       電子下游-手機製造     105
12      C22010           傳產-汽車     103
45      C23320       電子下游-電信服務      86
16      C23030  電子上游-IC-DRAM製造      49

[70 rows x 3 columns]


In [206]:
count_classi_y = pred_merge_code.groupby(['classi_code', '指數彙編分類', 'year']).size().reset_index(name='counts').sort_values(by=['year','counts'], ascending=False)
print(count_classi_y[count_classi_y['year']=='2023'])

print(count_classi_y[count_classi_y['year']=='2022'])

    classi_code          指數彙編分類  year  counts
63       C17020           傳產-生技  2023     506
543      C29020           傳產-其他  2023     485
119      C23010      電子上游-IC-設計  2023     388
479      C25010           傳產-營建  2023     295
39       C15010           傳產-電機  2023     238
..          ...             ...   ...     ...
431      C23410       電子下游-手機製造  2023      15
559      C30010         傳產-高爾夫球  2023      13
343      C23290       電子下游-數位相機  2023       9
367      C23320       電子下游-電信服務  2023       9
135      C23030  電子上游-IC-DRAM製造  2023       6

[70 rows x 4 columns]
    classi_code          指數彙編分類  year  counts
62       C17020           傳產-生技  2022     602
542      C29020           傳產-其他  2022     536
478      C25010           傳產-營建  2022     339
118      C23010      電子上游-IC-設計  2022     336
38       C15010           傳產-電機  2022     238
..          ...             ...   ...     ...
390      C23360        電子下游-掃描器  2022      14
430      C23410       電子下游-手機製造  2022      13
70       C1

In [207]:
# 將 merged(兩模型都有預測的個股)加上股票產業彙編碼
merged_merge_code = merged.merge(stock_info_dedup, on='stockid', how='left')
#print(merged_merge_code)

In [208]:
# 計算各產業數量(全，非僅包含預測相反者)
merge_count = merged_merge_code.groupby(['classi_code', '指數彙編分類']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
print(merge_count)

   classi_code          指數彙編分類  counts
7       C17020           傳產-生技   11502
67      C29020           傳產-其他   11068
14      C23010      電子上游-IC-設計    8069
59      C25010           傳產-營建    7488
4       C15010           傳產-電機    6479
..         ...             ...     ...
8       C18010         傳產-玻璃陶瓷     470
42      C23290       電子下游-數位相機     376
53      C23410       電子下游-手機製造     285
45      C23320       電子下游-電信服務     282
16      C23030  電子上游-IC-DRAM製造      94

[70 rows x 3 columns]


In [209]:
# 兩模型在各產業預測相反的比例 = 各產業不分時間預測相反總數量/兩模型都有預測的數量

# merge the df on 'classi_code'
ratio_merge = pd.merge(count_classi, merge_count, on=['classi_code', '指數彙編分類'], suffixes=('_c', '_m'))

# Calvulate the ratio and create new column
ratio_merge['ratio'] = ratio_merge['counts_c'] / ratio_merge['counts_m'] *100

# 依照占比降續排列
ratio_merge = ratio_merge.sort_values('ratio', ascending=False)
#print(len(ratio_merge[['classi_code','指數彙編分類', 'ratio']])) #70

In [210]:
# 計算預測相反比例前35高的產業(共70個產業)，電子上游、中游、下游、software 與傳產的數量
HighRatio35 = ratio_merge[['classi_code','指數彙編分類', 'ratio']][:35]

count_eeupstream = round(HighRatio35['指數彙編分類'].str.contains('電子上游').sum() / ratio_merge['指數彙編分類'].str.contains('電子上游').sum() *100, 2)
count_eemidstream = round(HighRatio35['指數彙編分類'].str.contains('電子中游').sum() / ratio_merge['指數彙編分類'].str.contains('電子中游').sum() *100, 2)
count_eedownstream = round(HighRatio35['指數彙編分類'].str.contains('電子下游').sum() / ratio_merge['指數彙編分類'].str.contains('電子下游').sum() *100, 2)
count_software = round(HighRatio35['指數彙編分類'].str.contains('軟體').sum() / ratio_merge['指數彙編分類'].str.contains('軟體').sum() *100, 2)
count_traditional =round( HighRatio35['指數彙編分類'].str.contains('傳產').sum() / ratio_merge['指數彙編分類'].str.contains('傳產').sum() *100, 2)
print('電子上游:', count_eeupstream, '電子中游:', count_eemidstream, '電子下游:', count_eedownstream, '軟體:', count_software, '傳產:', count_traditional)


電子上游: 76.92 電子中游: 40.0 電子下游: 50.0 軟體: 66.67 傳產: 47.62


In [211]:
# 針對電子上游產業觀察兩模型預測1和0的數量
stock_df = stock_info_dedup.dropna()
ee_upstream = stock_df[stock_df['指數彙編分類'].str.contains('電子上游')] #404

In [212]:
# 所有電子上游類別
uni_up = ee_upstream['指數彙編分類'].drop_duplicates()
print(uni_up)

139        電子上游-IC-通路
142         電子上游-連接元件
151     電子上游-PCB-材料設備
295      電子上游-LED及光元件
299        電子上游-IC-代工
303       電子上游-PCB-製造
310         電子上游-被動元件
312        電子上游-IC-封測
316        電子上游-IC-製造
317        電子上游-IC-其他
319       電子上游-DRAM銷售
336        電子上游-IC-設計
366    電子上游-IC-DRAM製造
Name: 指數彙編分類, dtype: object


In [213]:
# 電子上游-IC-DRAM製造 
# 計算夢幻、logit預測的0與1數目
logit_pred_eeupstream = pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-DRAM製造']['predict_labels'].sum()
dream_pred_eeupstream = pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-DRAM製造']['direction'].sum()

print('電子上游-IC-DRAM製造')
print('logit_prediction:', logit_pred_eeupstream, 'dream report prediction:', dream_pred_eeupstream)
 # all 49

電子上游-IC-DRAM製造
logit_prediction: 19.0 dream report prediction: 30.0


In [214]:
# 電子上游-IC-製造
logit_pred_ICmanufact = pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-製造']['predict_labels'].sum()
dream_pred_ICmanufact = pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-製造']['direction'].sum()

print('電子上游-IC-製造')
print('logit_prediction:', logit_pred_ICmanufact, 'dream report prediction:', dream_pred_ICmanufact)
#len( pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-製造']['direction']) # all 368


電子上游-IC-製造
logit_prediction: 71.0 dream report prediction: 297.0


In [215]:
# 電子上游-IC-代工
logit_pred_ICOEM = pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-代工']['predict_labels'].sum()
dream_pred_ICOEM = pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-代工']['direction'].sum()

print('電子上游-IC-代工')
print('logit_prediction:', logit_pred_ICOEM, 'dream report prediction:', dream_pred_ICOEM)

#len( pred_merge_code[pred_merge_code['指數彙編分類']=='電子上游-IC-代工']['direction']) # all 291

電子上游-IC-代工
logit_prediction: 52.0 dream report prediction: 239.0


In [216]:
# 意見分歧低
# 傳產-汽車
logit_pred_car = pred_merge_code[pred_merge_code['指數彙編分類']=='傳產-汽車']['predict_labels'].sum()
dream_pred_car = pred_merge_code[pred_merge_code['指數彙編分類']=='傳產-汽車']['direction'].sum()

print('傳產-汽車')
print('logit_prediction:', logit_pred_car, 'dream report prediction:', dream_pred_car)

傳產-汽車
logit_prediction: 66.0 dream report prediction: 37.0


In [217]:
# 電子下游-筆記型電腦
logit_pred_pc = pred_merge_code[pred_merge_code['指數彙編分類']=='電子下游-筆記型電腦']['predict_labels'].sum()
dream_pred_pc = pred_merge_code[pred_merge_code['指數彙編分類']=='電子下游-筆記型電腦']['direction'].sum()

print('電子下游-筆記型電腦')
print('logit_prediction:', logit_pred_pc, 'dream report prediction:', dream_pred_pc)

電子下游-筆記型電腦
logit_prediction: 101.0 dream report prediction: 92.0


In [218]:
# 計算兩模型預測相反，比例最高的前35個產業中，有多少是夢幻預測上漲(1)的數量大於 logit model

# Group by '指數彙編分類' and calculate the sum of 'predict_labels' and 'direction'
grouped = pred_merge_code.groupby('指數彙編分類')[['predict_labels', 'direction']].sum()

grouped['result'] = np.where(grouped['direction'] > grouped['predict_labels'], 1, np.where(grouped['direction'] == grouped['predict_labels'], 'even', 0))
HighRatio35_industry = HighRatio35['指數彙編分類']

# Filter the grouped DataFrame
grouped = grouped[grouped.index.isin(HighRatio35_industry)]

# Reindex the grouped DataFrame to match the order of HighRatio35_industry
grouped = grouped.reindex(HighRatio35_industry)

result_counts = grouped['result'].value_counts()

print(result_counts)
print(grouped)


result
1    19
0    16
Name: count, dtype: int64
                predict_labels  direction result
指數彙編分類                                          
電子上游-IC-DRAM製造            19.0       30.0      1
電子上游-IC-製造                71.0      297.0      1
電子上游-IC-代工                52.0      239.0      1
電子下游-光碟片                 184.0       95.0      0
軟體-其他                    213.0      384.0      1
電子下游-數位相機                 82.0       69.0      0
軟體-遊戲                    243.0      261.0      1
電子中游-NB與手機零組件            472.0      591.0      1
傳產-橡膠                    273.0      152.0      0
電子上游-IC-設計              1034.0     2060.0      1
電子中游-PC介面卡                72.0      142.0      1
傳產-生技                   1162.0     2933.0      1
電子上游-IC-封測               328.0      562.0      1
電子下游-安全監控                319.0      160.0      0
電子中游-其他                  501.0      460.0      0
電子中游-LCD-TFT面板           174.0       94.0      0
傳產-觀光                    700.0      660.0      0
電子下游-太陽能            

In [219]:
# 計算兩模型預測相反，比例最高的前35個產業中，有多少是夢幻預測上漲(1)的數量小於 logit model



In [220]:
# 電子上游 - 1 居多
print(grouped[grouped.index.str.contains('電子上游')])

# 傳產 - 0&1 各半
print(grouped[grouped.index.str.contains('傳產')])


                predict_labels  direction result
指數彙編分類                                          
電子上游-IC-DRAM製造            19.0       30.0      1
電子上游-IC-製造                71.0      297.0      1
電子上游-IC-代工                52.0      239.0      1
電子上游-IC-設計              1034.0     2060.0      1
電子上游-IC-封測               328.0      562.0      1
電子上游-LED及光元件             707.0      442.0      0
電子上游-IC-通路               332.0      489.0      1
電子上游-PCB-製造              679.0      663.0      0
電子上游-被動元件                442.0      703.0      1
電子上游-IC-其他               210.0      269.0      1
        predict_labels  direction result
指數彙編分類                                  
傳產-橡膠            273.0      152.0      0
傳產-生技           1162.0     2933.0      1
傳產-觀光            700.0      660.0      0
傳產-營建           1020.0     1595.0      1
傳產-航運            438.0      553.0      1
傳產-紙業            106.0      102.0      0
傳產-百貨            592.0      436.0      0
傳產-其他           1752.0     2157.0      1
傳產

In [221]:
# 電子上游準確性

# 營收真實發展方向
real_direction = df[['ddate', 'stockid', 'direction']]
print(real_direction)

         ddate  stockid  direction
0       202310     1101          1
1       202310     1102          1
2       202310     1103          1
3       202310     1104          1
4       202310     1108          1
...        ...      ...        ...
205195  201401     9951          1
205196  201401     9955          0
205197  201401     9958          0
205198  201401     9960          0
205199  201401     9962          0

[201965 rows x 3 columns]


In [239]:
Dream_report_class = Dream_report.merge(stock_info_dedup[['指數彙編分類', 'stockid']], on='stockid', how='left')
print(Dream_report_class.head())
print(len(Dream_report_class))

    ddate stockid  direction 指數彙編分類
0  202503    1213        0.0  傳產-食品
1  202503    1220        1.0  傳產-食品
2  202503    1225        1.0  傳產-食品
3  202503    1236        0.0  傳產-食品
4  202503    1256        1.0  傳產-食品
186845


In [238]:
logit_direction_class = logit_direction.merge(stock_info_dedup[['指數彙編分類', 'stockid']], on='stockid', how='left')
print(logit_direction_class.head())
print(len(logit_direction_class))

    ddate stockid  predict_labels 指數彙編分類
0  202310    1101             1.0  傳產-水泥
1  202310    1102             1.0  傳產-水泥
2  202310    1103             0.0  傳產-水泥
3  202310    1104             1.0  傳產-水泥
4  202310    1108             1.0  傳產-水泥
201965


In [240]:
# merge 3 dataframes and calculate the accuracy
df_DL = Dream_report_class.merge(logit_direction_class, on=['ddate', 'stockid', '指數彙編分類'], how='inner')
print(df_DL)

         ddate stockid  direction    指數彙編分類  predict_labels
0       202310    1101        1.0     傳產-水泥             1.0
1       202310    1102        1.0     傳產-水泥             1.0
2       202310    1103        1.0     傳產-水泥             0.0
3       202310    1104        1.0     傳產-水泥             1.0
4       202310    1108        1.0     傳產-水泥             1.0
...        ...     ...        ...       ...             ...
157161  201601    9951        1.0  傳產-汽車零組件             0.0
157162  201601    9955        0.0     傳產-其他             1.0
157163  201601    9958        0.0     傳產-鋼鐵             0.0
157164  201601    9960        0.0   傳產-高爾夫球             0.0
157165  201601    9962        0.0     傳產-鋼鐵             1.0

[157166 rows x 5 columns]


In [242]:
df_ABC = df_DL.merge(df[['ddate', 'stockid', 'direction']], on=['ddate', 'stockid'], suffixes=('_A', '_ABC'))
df_ABC

ValueError: You are trying to merge on object and int64 columns for key 'stockid'. If you wish to proceed you should use pd.concat

In [248]:
df_ABC = df_DL.merge(df[['stockid', 'ddate', 'direction']].astype({'stockid': str}), on=['stockid', 'ddate'], suffixes=('', '_real'))

print(df_ABC)

         ddate stockid  direction    指數彙編分類  predict_labels  direction_real
0       202310    1101        1.0     傳產-水泥             1.0               1
1       202310    1102        1.0     傳產-水泥             1.0               1
2       202310    1103        1.0     傳產-水泥             0.0               1
3       202310    1104        1.0     傳產-水泥             1.0               1
4       202310    1108        1.0     傳產-水泥             1.0               1
...        ...     ...        ...       ...             ...             ...
157161  201601    9951        1.0  傳產-汽車零組件             0.0               0
157162  201601    9955        0.0     傳產-其他             1.0               0
157163  201601    9958        0.0     傳產-鋼鐵             0.0               0
157164  201601    9960        0.0   傳產-高爾夫球             0.0               0
157165  201601    9962        0.0     傳產-鋼鐵             1.0               0

[157166 rows x 6 columns]
